# Getting started
## Local Installation
To install the last version of `yaeos` uploaded to PyPI simply do:

```shell
pip install yaeos
```
## Building from source
To build `yaeos` from source you can clone the repository and install it with 
pip. The system dependencies are:

- LAPACK
- Fortran compiler

To build and install do:

```shell
git clone https://github.com/ipqa-research/yaeos.git
cd python
pip install .
```

## Setting a Google Colab to use yaeos

To install `yaeos` on Google Colab you can do the following command to install 
the last version uploaded to PyPI:

In [ ]:
%pip install yaeos

On the other hand, if you want to install the `dev` version of `yaeos` in a
Google Colab and try out the new (and experimental) features you can execute
the following code:

>> Note: do not execute the next cell if you don't want the `dev` version.
>> use the previous cell command instead and skip the next one.

In [ ]:
# Installing the dev version into a google colab
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:
  !git clone https://github.com/ipqa-research/yaeos.git
  %cd yaeos
  !git checkout dev
  %pip install ./python
  %cd ..
  %rm -r yaeos

# Getting started

In [ ]:
import yaeos
import numpy as np

In [ ]:
# Number of components, for easier definition
nc = 2

# kij and lij matrices
k12 = 0.1
lij = kij = np.zeros((nc,nc))
kij[0,1] = kij[1,0] = k12

mixrule = yaeos.QMR(kij, lij)

# Critical constants
Tc = [320, 375]
Pc = [30, 45]
w = [0.0123, 0.045]

model = yaeos.PengRobinson76(Tc, Pc, w, mixrule)

n = [1.0, 0.2]
model.lnphi_vt(n, v=2.0, t=303.15, dn=True)

# Calculating an isotherm

In [ ]:
import yaeos as chadeos
import numpy as np
import matplotlib.pyplot as plt

nc = 2
n = [0.4, 0.6]
Tc = [190.564, 425.12]
Pc = [45.99, 37.96]
w = [0.0115478, 0.200164]

lij = kij = np.zeros((nc,nc))
mixrule = chadeos.QMR(kij, lij)
model = chadeos.PengRobinson76(Tc, Pc, w, mixrule)

V = 1.0
T = 303.150
ts = [150, 200, 300, 400]

for T in ts:
    vs = np.linspace(0.01, 5, 1000)
    ps = [
        model.pressure(n, V, T, dn=True, dt=True, dv=True)["P"] 
        for V in vs
    ]

    plt.plot(vs, ps)
    plt.xlim(0.01, None)
    plt.ylim(-100, 100)
plt.show()

# Flash PT

In [ ]:
import yaeos
import numpy as np

P, T = 60.0, 294.0

n = [0.4, 0.6]
tc = [190.564, 425.12]
pc = [45.99, 37.96]
w = [0.0115478, 0.200164]
model = yaeos.PengRobinson76(tc, pc, w)
model.flash_pt(n, P, T)["x"]

In [ ]:
ts = np.linspace(300, 400, 100, dtype=np.float64)
ps = np.linspace(10, 100, 100, dtype=np.float64)
betas = []

t = []
p = []

In [ ]:
for i in range(len(ts)):
    for j in range(len(ps)):
       flash = model.flash_pt(n, pressure=ps[j], temperature=ts[i])
       t.append(ts[i])
       p.append(ps[j])
       betas.append(flash["beta"])

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(t, p, c=betas)

## Modified Huron-Vidal MixingRule

In [ ]:
%%time
from yaeos import NRTL, MHV, PengRobinson76, yaeos_c, QMR
import numpy as np
import matplotlib.pyplot as plt

nc = 2
tc = [647.14, 513.92]
pc = [220.64, 61.48]
w =  [0.344, 0.649]

a = [[0, 3.458], [-0.801, 0]]
b = [[0, -586.1], [246.2, 0]]
c = [[0, 0.3], [0.3, 0]]

b = np.array(b)

ge_model = NRTL(a, b, c)
mixrule = MHV(ge_model, q=-0.53)

null_mixing = QMR(np.zeros((nc,nc)), np.zeros((nc,nc)))

model_stock = PengRobinson76(tc, pc, w, null_mixing)
model_mhv = PengRobinson76(tc, pc, w, mixrule)

colors = ["red", "green", "blue", "orange", "black"]
ts = np.linspace(50+273, 200+273, 9)
import time
for i, T in enumerate(ts):
    i=4
    xs = np.linspace(0.001, 0.999, 100)
    ys = []
    ps = []

    st = time.time()
    
    for x1 in xs:
        x = [x1, 1-x1]
        sat = model_mhv.saturation_pressure(x, T, "bubble")
        p, y = sat["P"], sat["y"]
        ps.append(p)
        ys.append(y[0])

    plt.plot(xs, ps, color=colors[i])
    plt.plot(ys, ps, color=colors[i])

# Phase envelope tracing

In [ ]:
import yaeos
import matplotlib.pyplot as plt
import numpy as np
import chemicals

In [ ]:
names = ["methane", "n-decane", "propane", "hexatriacontane"]
chems = []

Tc = []
Pc = []
w = []
zc = []
for c in names:
    chem = chemicals.CAS_from_any(c)
    Tc.append(chemicals.Tc(chem))
    Pc.append(chemicals.Pc(chem)/1e5)
    w.append(chemicals.acentric.omega(chem))
    zc.append(chemicals.Zc(chem))

print(Tc)
print(Pc)
print(w)
print(zc)

In [ ]:
z = np.ones(len(names))
z = z/z.sum()

models = [
    yaeos.PengRobinson76,
    yaeos.RKPR,
    yaeos.PengRobinson78,
    yaeos.SoaveRedlichKwong,
]

for m in models:
    if m == yaeos.RKPR:
        model = m(Tc, Pc, w, zc)
    else:
        model = m(Tc, Pc, w)
    T, P, Tcs, Pcs = model.phase_envelope_pt(
        z, kind="dew", T0=150, P0=0.01, max_points=350
        )
    plt.plot(T, P, label=model.name)
    plt.scatter(Tcs, Pcs, label=model.name + "-CP")

plt.legend()
plt.show()

In [ ]:
from ipywidgets import interact, fixed
import ipywidgets as widgets

from functools import cache

In [ ]:
%matplotlib inline
x = np.linspace(-10, 10,100)

def f(x, A, B, C):
    return A*x**2 + B*x + C

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
line, = ax.plot(x, f(x, A=1, B=1, C=1))

@cache
def run(i, x, n):
    plt.clf()
    z = np.ones(n)
    z[i] = x
    exclude = [i]
    mask = np.ones(z.shape, bool)
    mask[exclude] = False
    z[mask] = (1 - x)/2
    
    z = z/sum(z)
    print(z)

    models = [
        yaeos.PengRobinson76,
        # yaeos.RKPR,
        # yaeos.PengRobinson78,
        # yaeos.SoaveRedlichKwong,
    ]

    ax.clear()
    for m in models:
        if m == yaeos.RKPR:
            model = m(Tc, Pc, w, zc)
        else:
            model = m(Tc, Pc, w)
        T, P, Tcs, Pcs = model.phase_envelope_pt(
            z, kind="dew", T0=150, P0=0.01, max_points=1000
            )
        plt.plot(T, P, label=model.name)
        T, P, Tcs, Pcs = model.phase_envelope_pt(
            z, kind="bubble", T0=150, P0=0.01, max_points=1000
            )
        plt.plot(T, P, label=model.name, zorder=9)
    plt.legend()
    plt.show()
    
    ax.set_xlim(0, 1000)
    ax.set_ylim(0, 550)
    return T, P

@interact
def update(i=[i for i in range(len(Tc))], x=(0.0001, 0.9999, 0.01), n=fixed(len(Tc))):
    T, P = run(i, x, n)

# RKPR

In [ ]:
import yaeos
from ipywidgets import interact, fixed, FloatSlider
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import chemicals

In [ ]:
names = ["methane", "n-propane", "n-decane", "hexatriacontane"]
chems = []

Tc = []
Pc = []
w = []
zc = []
for c in names:
    chem = chemicals.CAS_from_any(c)
    Tc.append(chemicals.Tc(chem))
    Pc.append(chemicals.Pc(chem)/1e5)
    w.append(chemicals.acentric.omega(chem))
    zc.append(chemicals.Zc(chem))

In [ ]:
%matplotlib inline

x = np.linspace(-10, 10,100)
def f(x, A, B, C):
    return A*x**2 + B*x + C
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
line, = ax.plot(x, f(x, A=1, B=1, C=1))

z = np.ones(len(names))
z = z/z.sum()
sliders = []
for i in range(len(names)):
    sliders.append(
        FloatSlider(
            value=0.1, min=1e-5, max=15, step=0.01, 
            description=names[i]
        )
    )

def run(d1, d2, d3, d4):
    print(d1, d2, d3, d4)
    ax.clear()
    model = yaeos.RKPR(Tc, Pc, w, zc, delta_1=[d1, d2, d3, d4])
    T, P, Tcs, Pcs = yaeos.yaeos_c.pt2_phase_envelope(
                model.id, z, kind="dew", t0=200, p0=0.01, max_points=500
                )
    ax.plot(T, P)
    T, P, Tcs, Pcs = yaeos.yaeos_c.pt2_phase_envelope(
                model.id, z, kind="bubble", t0=150, p0=10.0, max_points=500
                )
    ax.plot(T, P)
    ax.set_xlim(100, 1000)
    ax.set_ylim(1, 150)
    return T, P

def update(**kwargs):
    T, P = run(**kwargs)

interact(update, d1=sliders[0], d2=sliders[1], d3=sliders[2], d4=sliders[3])